In [ ]:
'''
Mohamed Abdelkoddous - Matthieu Rioual
03/05/20
CPE Lyon - 4ETI
Quantum Programming
Project #2
'''
 
#!usr/bin/python
    
# importing qiskit for quantum programming
from qiskit import *
# importing math for pi and sqrt
import math
 
def circuit_definition(input_state, measurement_settings):
   
    # initialize a quantum circuit with two quantum registers and two classical registers
    qc = QuantumCircuit(2,2)
   
    # initialize the state of both qbit
    qc.initialize(input_state, [0,1])
   
    # first qbit
    if measurement_settings[0] == 'X':
        # we transforme in X basis
        qc.h(0)
 
    # second qbit
    if measurement_settings[1] == 'W':
        # (Z+X)/sqrt(2)
        qc.u3(-math.pi/4, 0, 0, 1)                
    else:
        # (Z-X)/sqrt(2)
        qc.u3(math.pi/4, 0, 0, 1)                
       
    # measurement on classical qbits
    qc.measure(0,0)
    qc.measure(1,1)
   
    # send back the generated circuit
    return qc
 
def circuit_execution(input_state, file_name1, file_name2, backend_definition):
    result = []
   
    # mapping which associate random numbers with measurement settings
    dict={'00':'ZW', '01':'ZV', '10':'XW', '11':'XV'}
   
    # open the files in which there are random generated bits
    file_first_qb = open(file_name1, "r")
    file_second_qb = open(file_name2, "r")
   
    emulator = Aer.get_backend(backend_definition)
   
    # launch the loop that will provides us datas in order to treat them
    for i in range(0, 2**15):
 
        # read the files
        sequence1 = file_first_qb.read(1)
        sequence2 = file_second_qb.read(1)
       
        # first we choose the measurement settings thanks to the bits and the correspondance dictionary we created above
        choosen_setting = dict.get(sequence1+sequence2)
       
        # then we create the circuit corresponding to the classical measure of the two initial qbits 
        # which underwent 2 randomized mesuring gates 
        circuit = circuit_definition(input_state, choosen_setting)
       
        # we execute the given circuit to get the result of one particular configuration
        job = execute( circuit, emulator, shots=1).result().get_counts()
               
        # that we stock in a tuple with the configuration
        result.append((choosen_setting, next(iter(job))))
       
    return result
       
def S_parameter(input_state, file_name1, file_name2, backend_definition):
       
    # get the results in a 2^15 tuple list
    datas = circuit_execution(input_state, file_name1, file_name2, backend_definition)
   
    # S is the variable we are looking for demonstrating the bell's ineqation violation
    S = 0
    A = ["ZW","XW","ZV","XV"]
    B = ["00","01","10","11"]
    C = [[] for k in range(4)]
   
    # we count every occurrence of the 4 possibilities (00-01-10-11) and the 4 directions (ZW-ZV-XW-XV)
    for i in range(len(A)):
        for j in range(len(B)):
            C[i].append(datas.count((A[i], B[j])))
 
   # table of probabilities of the 4 possibilities (00-01-10-11) and the 4 directions (ZW-ZV-XW-XV)
    P=[]
    for i in range(4):
        for j in range(4):
           P.append(C[i][j]/sum(C[i]))
        

    # E=P(00)-P(01)-P(10)+P(11)
    E1, E2, E3, E4 = P[0]-P[1]-P[2]+P[3], P[4]-P[5]-P[6]+P[7], P[8]-P[9]-P[10]+P[11], P[12]-P[13]-P[14]+P[15]
   
    # S=<ZW>+<ZV>+<XW>-<XV>
    S = E1 + E3 + E2 - E4
   
    # results are rounded up to 3 digits for the summary display
    P = [round(p, 3) for p in P]
    E1, E2, E3, E4 = round(E1,3), round(E2,3), round(E3,3), round(E4,3)
 
    # summary display
    ligne1 = "     P(00)  P(11)  P(01)  P(10)  <AB>\n"
    ligne2 = "ZW "+"  "+str(P[0])+"  "+str(P[1])+"  "+str(P[2])+"  "+str(P[3])+"  "+str(E1)+"\n"
    ligne3 = "ZV "+"  "+str(P[4])+"  "+str(P[5])+"  "+str(P[6])+"  "+str(P[7])+"  "+str(E2)+"\n"
    ligne4 = "XW "+"  "+str(P[8])+"  "+str(P[9])+"  "+str(P[10])+"  "+str(P[11])+"  "+str(E3)+"\n"
    ligne5 = "XV "+"  "+str(P[12])+"  "+str(P[13])+"  "+str(P[14])+"  "+str(P[15])+"  "+str(E4)+"\n"
    summary = ligne1 + ligne2 + ligne3 + ligne4 + ligne5
   
    return S, summary
 
# S_papameter arguments
input_state = [1 / math.sqrt(2),
               0,
               0,
               1 / math.sqrt(2)]
file_name1 = "random_settings_a.txt"
file_name2 = "random_settings_b.txt"
backend_definition = 'qasm_simulator'

# execution of S_papameter
S_final, Summary = S_parameter(input_state, file_name1, file_name2, backend_definition)

# display of the results
print("Le parametre S trouvé est de %f." % S_final)
print(Summary)